In [1]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/Shared/auto/07p/python")
# sys.path.insert(0, "/Users/ielyaascloete/auto/07p/python")
from auto import *


In [2]:
cl

<function AUTOCommands.cl>

In [3]:
import sys
print(sys.version)

2.7.14 |Anaconda custom (64-bit)| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [4]:
Eq = run(e='PLC_final',c='PLC_final.eq')

Starting PLC_final ...

  BR    PT  TY  LAB     V_plc        L2-NORM         Kplc        PAR(3)        PAR(4)        PAR(5)        PAR(6)        PAR(7)     
   1     1  EP    1   1.00000E-10   3.92549E+00   1.10000E-01   0.00000E+00   8.05296E-02   3.86101E+00   7.03950E-01   1.00000E-10
   1    71  HB    2   5.25489E-02   3.49999E+00   1.10000E-01   0.00000E+00   8.75240E-02   3.44166E+00   6.30189E-01   1.01857E-02
   1   412  HB    3   4.54840E-01   3.61406E-01   1.10000E-01   0.00000E+00   1.43270E-01   2.29832E-01   1.91818E-01   1.43077E-01
   1  1413  EP    4   1.00008E+01   3.15968E+00   1.10000E-01   0.00000E+00   1.43270E-01   1.71383E-01   1.91817E-01   3.14593E+00

 Total Time    0.130E+00
PLC_final ... done


Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [5]:
PO1 = run(Eq('HB1'),c ='PLC_final.per',SP=['BP0'],DS=1e-3)

Starting PLC_final ...

  BR    PT  TY  LAB     V_plc        L2-NORM        PERIOD         Kplc        PAR(3)        PAR(4)        PAR(5)        PAR(6)     
   2    17  LP    5   5.25176E-02   3.49142E+00   9.00557E+01   1.10000E-01   0.00000E+00   9.24978E-02   3.44697E+00   6.31442E-01
   2   228  PD    6   5.39148E-02   3.53747E+00   2.11108E+02   1.10000E-01   0.00000E+00   5.44264E-01   3.58622E+00   5.33465E-01
   2   344  PD    7   1.66652E-01   2.83885E+00   3.39096E+01   1.10000E-01   0.00000E+00   6.57658E-01   2.95343E+00   1.65059E-01
   2   646  LP    8   4.54840E-01   3.61406E-01   7.88039E+00   1.10000E-01   0.00000E+00   1.43272E-01   2.29833E-01   1.91818E-01
   2   737  LP    9   6.63345E-01   4.29216E-01   2.22825E+01   1.10000E-01   0.00000E+00   3.48734E-01   4.61775E-01   1.29150E-01
 ** On entry to DGEBAL parameter number  3 had an illegal value
PLC_final ... done


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'LPPR' Limit point of periodic orbit
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    print("Acquire the b. file")
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()
    
    
    print("Read file")
    """Read the file from the beginning of the interesting part (depends on its type)"""
    
    for i in range(len(content)):
    
        if "PT" in content[i]:
            init = i
            break



            
    content_csv = [[el for el in content[init].split(' ') if len(el) > 0 and el != '\n']]
    print(content_csv)
        
    
        
    print("Rename branch")
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    
    
    print("Split content")
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
        
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [8]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

Relabeling done
Saving to b.Eq, s.Eq, and d.Eq ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'Kplc', 'PAR(3)', 'PAR(4)', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(31)', 'PAR(32)', 'PAR(33)', 'PAR(34)', 'PAR(80)']]
Rename branch
Split content


In [9]:
df_PR = auto_to_csv(PO1,'PO1','PR')

Relabeling done
Saving to b.PO1, s.PO1, and d.PO1 ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'PERIOD', 'Kplc', 'PAR(3)', 'PAR(4)', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(31)', 'PAR(32)', 'PAR(33)', 'PAR(34)', 'PAR(80)']]
Rename branch
Split content


In [10]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [11]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(4)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(4)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(31)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [12]:
Is_1 = run(e='PLC_final',c='PLC_final.per',dat='L3',ISW=1,PAR={1:0.3},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_2 = run(e='PLC_final',c='PLC_final.per',dat='L3', DS='-',ISW=1,SP=['BP0'],PAR={1:0.3},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is = Is_1 + Is_2

Starting PLC_final ...

  BR    PT  TY  LAB     V_plc        L2-NORM        PERIOD         Kplc        PAR(3)        PAR(4)        PAR(5)        PAR(6)     
   1     1  EP    1   3.00000E-01   1.30475E+00   8.24459E+01   1.10000E-01   0.00000E+00   7.63310E-01   1.94439E+00   2.75722E-01
   1   446  LP    2   5.70603E-01   8.44898E-01   6.55600E+01   1.10000E-01   0.00000E+00   7.80652E-01   1.35423E+00   2.46896E-01
   1   478  BP    3   5.70603E-01   8.19580E-01   6.82858E+01   1.10000E-01   0.00000E+00   7.78496E-01   1.32459E+00   2.61653E-01
   1   480  BP    4   5.70603E-01   8.19322E-01   6.83356E+01   1.10000E-01   0.00000E+00   7.78455E-01   1.32409E+00   2.61907E-01
   1   482  BP    5   5.70603E-01   8.18894E-01   6.84225E+01   1.10000E-01   0.00000E+00   7.78383E-01   1.32321E+00   2.62348E-01
   1   491  BP    6   5.70603E-01   8.16184E-01   6.89801E+01   1.10000E-01   0.00000E+00   7.77430E-01   1.31540E+00   2.65271E-01
   1   528  LP    7   5.70603E-01   7.75936E-01   6

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


   1   559  PD    2   1.92538E-01   1.75903E+00   9.69554E+01   1.10000E-01   0.00000E+00   7.20734E-01   2.37554E+00   3.05663E-01
 ** On entry to DGEBAL parameter number  3 had an illegal value
PLC_final ... done


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [13]:
Is_3 = run(e='PLC_final',c='PLC_final.per',dat='L6',ISW=1,PAR={1:0.6},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=100)
Is_4 = run(e='PLC_final',c='PLC_final.per',dat='L6', DS='-',ISW=1,SP=['BP0'],PAR={1:0.6},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=100)
Is2 = Is_3 + Is_4

Starting PLC_final ...

  BR    PT  TY  LAB     V_plc        L2-NORM        PERIOD         Kplc        PAR(3)        PAR(4)        PAR(5)        PAR(6)     
   1     1  EP    1   6.00000E-01   7.98651E-01   6.71832E+01   1.10000E-01   0.00000E+00   7.77889E-01   1.30909E+00   2.45884E-01
   1   530  PD    2   6.03114E-01   7.64795E-01   7.00336E+01   1.10000E-01   0.00000E+00   7.74716E-01   1.27507E+00   2.60544E-01
   1   781  BP    3   6.03114E-01   7.49859E-01   7.22496E+01   1.10000E-01   0.00000E+00   7.72355E-01   1.25398E+00   2.71723E-01
   1   808  BP    4   6.03114E-01   7.48659E-01   7.25841E+01   1.10000E-01   0.00000E+00   7.71790E-01   1.25013E+00   2.73369E-01
   1   819  BP    5   6.03114E-01   7.47892E-01   7.26770E+01   1.10000E-01   0.00000E+00   7.71265E-01   1.24752E+00   2.73915E-01
   1   821  BP    6   6.03114E-01   7.47684E-01   7.26845E+01   1.10000E-01   0.00000E+00   7.71119E-01   1.24689E+00   2.73986E-01
   1   830  BP    7   6.03114E-01   7.45722E-01   7

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


   1  1327  LP    2   5.70603E-01   7.98769E-01   7.87231E+01   1.10000E-01   0.00000E+00   7.81695E-01   1.37107E+00   2.38955E-01
   1  1331  LP    3   5.70603E-01   7.98798E-01   7.87562E+01   1.10000E-01   0.00000E+00   7.81698E-01   1.37127E+00   2.38863E-01
   1  1510  PD    4   5.70603E-01   8.02679E-01   8.13821E+01   1.10000E-01   0.00000E+00   7.82721E-01   1.38847E+00   2.31073E-01
   1  1705  LP    5   5.70603E-01   8.08341E-01   8.50156E+01   1.10000E-01   0.00000E+00   7.83787E-01   1.41002E+00   2.21746E-01
   1  1795  LP    6   5.70603E-01   7.96745E-01   8.89697E+01   1.10000E-01   0.00000E+00   7.83544E-01   1.40609E+00   2.23411E-01
   1  1822  LP    7   5.70603E-01   7.88278E-01   9.00870E+01   1.10000E-01   0.00000E+00   7.83158E-01   1.39756E+00   2.27081E-01
   1  2026  LP    8   5.70603E-01   7.00555E-01   9.29895E+01   1.10000E-01   0.00000E+00   7.52811E-01   1.22957E+00   2.53127E-01
   1  2073  LP    9   5.70602E-01   6.59505E-01   8.81923E+01   1.10000E-01 

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [ ]:
Is_5 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per191',ISW=1,PAR={1:0.191},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_6 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per191', DS='-',ISW=1,SP=['BP0'],PAR={1:0.191},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is3 = Is_5 + Is_6

In [ ]:
#Is_3 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per3',ISW=1,PAR={1:0.3},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
#Is_4 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per3', DS='-',ISW=1,SP=['BP0'],PAR={1:0.3},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
#Is2 = Is_3 + Is_4

In [ ]:
Is_5 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per6',ISW=1,PAR={1:0.6},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is_6 = run(e='PKC_HR_IPR',c='PKC_HR_IPR.per',dat='per6', DS='-',ISW=1,SP=['BP0'],PAR={1:0.6},ISP=2,EPSL= 1e-12, EPSU = 1e-12,NTST=200)
Is3 = Is_5 + Is_6

In [14]:
df_Is = auto_to_csv(Is,'Is','IS')

Relabeling done
Saving to b.Is, s.Is, and d.Is ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'PERIOD', 'Kplc', 'PAR(3)', 'PAR(4)', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(31)', 'PAR(32)', 'PAR(33)', 'PAR(34)', 'PAR(80)']]
Rename branch
Split content


In [15]:
df_Is2 = auto_to_csv(Is2,'Is2','IS2')

Relabeling done
Saving to b.Is2, s.Is2, and d.Is2 ... done
Acquire the b. file
Read file
[['0', 'PT', 'TY', 'LAB', 'V_plc', 'L2-NORM', 'PERIOD', 'Kplc', 'PAR(3)', 'PAR(4)', 'PAR(5)', 'PAR(6)', 'PAR(7)', 'PAR(31)', 'PAR(32)', 'PAR(33)', 'PAR(34)', 'PAR(80)']]
Rename branch
Split content


In [ ]:
df_Is3 = auto_to_csv(Is3,'Is3','IS3')

In [17]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(4)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(4)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(31)'],
                  name='Periodic Orbit',
                   mode='lines')
tr_is = Scatter(x=df_Is['V_plc'],
                  y=df_Is['PAR(4)'],
                  name='Isola',
                  mode='lines')
tr_is_min = Scatter(x=df_Is['V_plc'],
                  y=df_Is['PAR(31)'],
                  name='Isola',
                  mode='lines')
tr_is2 = Scatter(x=df_Is2['V_plc'],
                  y=df_Is2['PAR(4)'],
                  name='Isola',
                  mode='lines')
tr_is2_min = Scatter(x=df_Is2['V_plc'],
                  y=df_Is2['PAR(31)'],
                  name='Isola',
                  mode='lines')
# tr_is3 = Scatter(x=df_Is3['V_plc'],
#                   y=df_Is3['PAR(5)'],
#                   name='Isola',
#                   mode='lines')
# tr_is3_min = Scatter(x=df_Is3['V_plc'],
#                   y=df_Is3['PAR(32)'],
#                   name='Isola',
#                   mode='lines')
# data = [trace_eq,trace_per,trace_per_min,tr_is,tr_is_min,tr_is2,tr_is2_min,tr_is3,tr_is3_min]
data = [trace_eq,trace_per,trace_per_min,tr_is,tr_is_min,tr_is2,tr_is2_min]
# data = [trace_eq,trace_per,trace_per_min,tr_is,tr_is_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
HB12P = run(Eq('HB1'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[1,3],DS=1e-3)

In [ ]:
HB12Pm = run(Eq('HB1'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[1,3],DS=-1e-3)

In [ ]:
HB22P = run(Eq('HB2'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[1,3],DS=1e-3)

In [ ]:
HB22Pm = run(Eq('HB2'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[1,3],DS=-1e-3)

In [ ]:
SNP2P = run(PO1('LP3'),c='PKC_HR_IPR.per',ISW=2,ICP=[1,2,11],DS=1e-4)
SNP2P1 = run(SNP2P('EP1'))    

In [ ]:
df_HB12P = auto_to_csv(HB12P,'HB12P','HB_1')
df_HB22P = auto_to_csv(HB22P,'HB22P','HB_2')
df_PD2P1 = auto_to_csv(PD2P1,'PD2P1','PD_1')
df_SNP2P1 = auto_to_csv(SNP2P1,'SNP2P1','SNP_1')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'LPPR' Limit point of periodic orbit
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    print("Acquire the b. file")
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()
    
    
    print("Read file")
    """Read the file from the beginning of the interesting part (depends on its type)"""
    
    for i in range(len(content)):
    
        if "PT" in content[i]:
            init = i
            break



            
    content_csv = [[el for el in content[init].split(' ') if len(el) > 0 and el != '\n']]
    print(content_csv)
        
    
        
    print("Rename branch")
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    
    
    print("Split content")
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
        
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [ ]:
df_eq = auto_to_csv(slice1,'slice1','EQ')

In [ ]:
df_PR = auto_to_csv(slicePO,'slicePO','PR')

In [ ]:
trace_eq = Scatter(x=df_eqs[8]['ct'],
                  y=df_eqs[8]['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(8)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='Agonist'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_per1[1:-1].to_csv('per1.csv')

In [ ]:
df_per2[1:-1].to_csv('per2.csv')

In [ ]:
df_per2[1:-1]